## HW#1: Extreme Rainfall in Singapore 

```{admonition} Objectives
:class: tip

This homework will help you gain a better understanding in terms of how to:
* Fit Generalized Extreme Value (GEV) distribution 
* Estimate the return period and return level of extreme rainfall

Happy coding!
```

```{admonition} Submission Guide

Deadline: **Sunday, 31st October 2021** 
(Note: Late submissions will not be accepted). 

Please upload your solutions to LumiNUS in a Jupyter Notebook format with the name "Homework1_StudentID.ipynb". Make sure to write down your student ID and full name in the cell below. 

For any questions, feel free to contact Prof. Xiaogang HE ([hexg@nus.edu.sg](mailto:hexg@nus.edu.sg)), Huimin WANG ([wanghuimin@u.nus.edu](mailto:wanghuimin@u.nus.edu)) or Zhanwei LIU ([liuzhanwei@u.nus.edu](mailto:liuzhanwei@u.nus.edu)).

```

In [1]:
### Fill your student ID and full name below.

# Student ID:
# Full name:

**Data**:
You will need to use the histrocial (1981-2020) daily total rainfall at Singapore's Changi station for this homework. 
You can create a DataFrame using Pandas by reading file "../../assets/data/Changi_daily_rainfall.csv".

### Q1: Fit the GEV distribution

Find the annual maximum rainfall based on the daily rainfall. This will result in a data set of 40 values - one per year. Fit the GEV distribution to the time series of annual maximum rainfall. To do this, estimate the GEV parameters using (i) Maximum Likelihood and (ii) L-Moments, respectively. Based on your results, discuss whether extreme rainfall in Singapore is bounded above or not. (Details on fitting a GEV distribution can be found in the [Scipy tutorial](https://xiaoganghe.github.io/python-climate-visuals/chapters/data-analytics/scipy-basic.html)). (40 marks)

In [2]:
from scipy.stats import genextreme
from scipy.optimize import fsolve
import numpy as np
import pandas as pd
import math

# Calculate L-moments based on samples
def samlmom3(sample):
    """
    samlmom3 returns the first three L-moments of samples
    sample is the 1-d array
    n is the total number of the samples, j is the j_th sample
    """
    n = len(sample)
    sample = np.sort(sample)[::-1]
    b0 = np.mean(sample)
    b1 = np.array([(n - j - 1) * sample[j] / n / (n - 1)
                   for j in range(n)]).sum()
    b2 = np.array([(n - j - 1) * (n - j - 2) * sample[j] / n / (n - 1) / (n - 2)
                   for j in range(n - 1)]).sum()
    lmom1 = b0
    lmom2 = 2 * b1 - b0
    lmom3 = 6 * (b2 - b1) + b0

    return lmom1, lmom2, lmom3

def pargev_fsolve(lmom):
    """
    pargev_fsolve estimates the parameters of the Generalized Extreme Value 
    distribution given the L-moments of the data
    """
    lmom_ratios = [lmom[0], lmom[1], lmom[2]/lmom[1]]
    f = lambda x, t: 2 * (1 - 3**(-x))/(1 - 2**(-x)) - 3 - t 
    G = fsolve(f, 0.01, lmom_ratios[2])
    para3 = G
    GAM = math.gamma(1 + G)
    para2 = lmom_ratios[1] * G / (GAM * (1 - 2 ** -G))
    para1 = lmom_ratios[0] - para2 * (1 - GAM) / G
    return para1[0], para2[0], para3[0]


In [3]:
df = pd.read_csv('../../assets/data/Changi_daily_rainfall.csv', 
                 index_col=0, header=0, parse_dates=True)
samples = df.resample('Y').max().values.flatten()
shp_LME, loc_LME, sca_LME = genextreme.fit(samples)

LMM = samlmom3(samples)
loc_LMM, sca_LMM, shp_LMM = pargev_fsolve(LMM)

print('Maximum Likelihood: loc_LME={}, sca_LME={}, shp_LME={}'.format(loc_LME, sca_LME, shp_LME))
print('L-Moments: loc_LMM={}, sca_LMM={}, shp_LMM={}'.format(loc_LMM, sca_LMM, shp_LMM))

Maximum Likelihood: loc_LME=101.1678211072257, sca_LME=39.24690874747871, shp_LME=0.02668390757775726
L-Moments: loc_LMM=101.88198201624115, sca_LMM=43.00458155155795, shp_LMM=0.08500214105957563


In [4]:
# According to the estimated parameters, both shp_LME and shp_LMM are positive. 
# Therefore, extreme rainfall distribution in Singapore is characterized as “heavy” tailed
# that is unbounded above. （Refer to Slide 19 of Week 10's lecture.）

### Q2: Determine the return level

Based on the estimated GEV parameters using L-Moments in Q1, determine the return level (i.e., magnitude of extreme rainfall) of a 20-year, 50-year and 100-year rainfall event. (30 marks).

In [5]:
def get_return_level(return_period, ppf):
    """
    calculate return level using ppf given the return period.
    
    """
    
    prob = 1 - 1 / return_period
    return_level = ppf(prob)
    
    return return_level

LMMGEV = genextreme(shp_LMM, loc=loc_LMM, scale=sca_LMM)

level20 = get_return_level(20, LMMGEV.ppf)
level50 = get_return_level(50, LMMGEV.ppf)
level100 = get_return_level(100, LMMGEV.ppf)

print('The return level:')
print('20-year: {} mm'.format(level20))
print('50-year: {} mm'.format(level50))
print('100-year: {} mm'.format(level100))

The return level:
20-year: 214.76504059731164 mm
50-year: 244.69311978158987 mm
100-year: 265.6165801276135 mm


### Q3: Estimate the return period

Based on the estimated GEV parameters using L-Moments in Q1, what is the return period if daily rainfall is 300 mm? (30 marks)

In [6]:
def get_return_period(return_level, cdf):
    """
    calculate return period using CDF given the return level.
    
    """
    
    prob = cdf(return_level)
    return_period = 1 / (1 - prob)
    
    return return_period

return_period = get_return_period(300, LMMGEV.cdf)
print('Return period of 300 mm is : {} year'.format(return_period))

Return period of 300 mm is : 346.327754435381 year
